In [1]:
import jams
import glob
import os
import librosa

In [2]:
jampaths = glob.glob("./jams/*.jams")

In [3]:
jampaths.sort()

In [4]:
jampaths

['./jams/BN1-129-Eb_comp.jams',
 './jams/BN1-147-Gb_comp.jams',
 './jams/BN2-131-B_comp.jams',
 './jams/BN2-166-Ab_comp.jams',
 './jams/BN3-119-G_comp.jams',
 './jams/BN3-154-E_comp.jams',
 './jams/Funk1-114-Ab_comp.jams',
 './jams/Funk1-97-C_comp.jams',
 './jams/Funk2-108-Eb_comp.jams',
 './jams/Funk2-119-G_comp.jams',
 './jams/Funk3-112-C#_comp.jams',
 './jams/Funk3-98-A_comp.jams',
 './jams/Jazz1-130-D_comp.jams',
 './jams/Jazz1-200-B_comp.jams',
 './jams/Jazz2-110-Bb_comp.jams',
 './jams/Jazz2-187-F#_comp.jams',
 './jams/Jazz3-137-Eb_comp.jams',
 './jams/Jazz3-150-C_comp.jams',
 './jams/Rock1-130-A_comp.jams',
 './jams/Rock1-90-C#_comp.jams',
 './jams/Rock2-142-D_comp.jams',
 './jams/Rock2-85-F_comp.jams',
 './jams/Rock3-117-Bb_comp.jams',
 './jams/Rock3-148-C_comp.jams',
 './jams/SS1-100-C#_comp.jams',
 './jams/SS1-68-E_comp.jams',
 './jams/SS2-107-Ab_comp.jams',
 './jams/SS2-88-F_comp.jams',
 './jams/SS3-84-Bb_comp.jams',
 './jams/SS3-98-C_comp.jams']

In [5]:
# i=2
# jampath = jampaths[i]
for jampath in jampaths:
    piece, tempo, key = os.path.basename(jampath).split('_')[0].split('-')
    tempo = float(tempo)
    beat_step = 60/tempo
    tran_semi = librosa.note_to_midi(key) % 12
    prog = piece[-1]
    print (piece, tempo, beat_step, key, tran_semi)

    jam = jams.load(jampath)
    if len(jam.search(namespace='beat_position')) == 0: 
        ann_beat = jams.Annotation(
            namespace='beat_position', time=0, 
            duration=jam.file_metadata.duration
        )

        if prog == '1':
            num_measure = 12
        else:
            num_measure = 16

        for i in range(num_measure * 4):
            measure = int(i/4) + 1 # measure number starts with one, not zero
            time = i * beat_step
            value = {
                "position": (i%4) + 1,
                "measure": measure,
                "num_beats": 4,
                "beat_units": 4
            }
            ann_beat.append(time=time, duration=0, value=value)
            ann_tempo = jams.Annotation(namespace='tempo', time=0, duration=jam.file_metadata.duration)
            ann_tempo.append(time=0, duration=jam.file_metadata.duration, value=tempo, confidence=1.0)

        ann_chord = jams.Annotation(
            namespace='chord', time=0, 
            duration=jam.file_metadata.duration
        )

        prog1_list = ['C:maj', 'F:maj', 'C:maj', 'G:maj', 'F:maj', 'C:maj']
        prog2_list = ['F:min', 'Bb:7', 'Eb:maj', 'Ab:maj','D:hdim7','G:7', 'C:min', 
                      'F:min', 'Bb:7', 'Eb:maj', 'Ab:maj','D:hdim7','G:7', 'C:min']
        prog3_list = ['C:maj', 'G:maj', 'A:min', 'E:min', 'F:maj', 'C:maj','F:maj', 'G:maj',
                      'C:maj', 'G:maj', 'A:min', 'E:min', 'F:maj', 'C:maj','F:maj', 'G:maj']

        chord_change = []
        if prog is '1':
            prog_list = prog1_list
            for beat in ann_beat:
                if beat.value['measure'] in [1,5,7,9,10,11] and beat.value['position'] == 1:
                    chord_change.append(beat.time)        
        if prog is '2':
            prog_list = prog2_list
            for beat in ann_beat:
                if beat.value['measure'] not in [8, 16] and beat.value['position'] == 1:
                    chord_change.append(beat.time)
        if prog is '3':
            prog_list = prog3_list
            for beat in ann_beat:
                if beat.value['position'] == 1:
                    chord_change.append(beat.time)
        chord_change.sort()

        for i, t in enumerate(chord_change):
            if i==len(chord_change)-1:
                duration = jam.file_metadata.duration - t
            else:
                duration = chord_change[i+1] - t

            chord = prog_list[i]
            root_in_c, quality = chord.split(':')
            root_in_key = librosa.midi_to_note(librosa.note_to_midi(root_in_c) + tran_semi)[:-1]
            transposed_chord = ':'.join([root_in_key, quality])
            ann_chord.append(time=t, duration=duration, value=transposed_chord)

        jam.annotations.append(ann_beat)
        jam.annotations.append(ann_tempo)
        jam.annotations.append(ann_chord)
        jam.save(jampath)
        

BN1 129.0 0.46511627906976744 Eb 3
BN1 147.0 0.40816326530612246 Gb 6
BN2 131.0 0.4580152671755725 B 11
BN2 166.0 0.3614457831325301 Ab 8
BN3 119.0 0.5042016806722689 G 7
BN3 154.0 0.38961038961038963 E 4
Funk1 114.0 0.5263157894736842 Ab 8
Funk1 97.0 0.6185567010309279 C 0
Funk2 108.0 0.5555555555555556 Eb 3
Funk2 119.0 0.5042016806722689 G 7
Funk3 112.0 0.5357142857142857 C# 1
Funk3 98.0 0.6122448979591837 A 9
Jazz1 130.0 0.46153846153846156 D 2
Jazz1 200.0 0.3 B 11
Jazz2 110.0 0.5454545454545454 Bb 10
Jazz2 187.0 0.32085561497326204 F# 6
Jazz3 137.0 0.43795620437956206 Eb 3
Jazz3 150.0 0.4 C 0
Rock1 130.0 0.46153846153846156 A 9
Rock1 90.0 0.6666666666666666 C# 1
Rock2 142.0 0.4225352112676056 D 2
Rock2 85.0 0.7058823529411765 F 5
Rock3 117.0 0.5128205128205128 Bb 10
Rock3 148.0 0.40540540540540543 C 0
SS1 100.0 0.6 C# 1
SS1 68.0 0.8823529411764706 E 4
SS2 107.0 0.5607476635514018 Ab 8
SS2 88.0 0.6818181818181818 F 5
SS3 84.0 0.7142857142857143 Bb 10
SS3 98.0 0.6122448979591837 C 0
